In [ ]:
# !pip install --upgrade pip

In [ ]:
# !pip install gensim

In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import emoji
from nltk.corpus import stopwords, wordnet
import time
from pathlib import Path

from collections import Counter
from spellchecker import SpellChecker
import os

from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
from nltk.stem import WordNetLemmatizer
from ftfy import fix_encoding
import spacy
from gensim.models import Word2Vec

#nlp = spacy.load("en_core_web_sm")

pd.set_option('display.max_colwidth', None)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


In [6]:

current_dir = os.getcwd()
csv_path  = os.path.abspath(os.path.join(current_dir, '../../../data/twitter_hate-speech/train_cleaned.csv'))
dict_path = os.path.abspath(os.path.join(current_dir, '../nasiba/data/frequency_dictionary_en_82_765.txt'))
df_cleaned = pd.read_csv(csv_path, encoding='utf-8')



df_cleaned=df_cleaned[df_cleaned['tweet_cleaned'].notna()]
df_cleaned.head(1)

,id,label,tweet,tweet_cleaned,user_handle,hashtags,emojis
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,father selfish drag kid run,1,['#run'],NaN


#### 1. Bag of Words (BoW)

# Bag of Words

Die Kernintuition hinter dem Bag of Words (BoW)-Modell ist, dass unser Algorithmus, wenn er ähnliche Wörter mit ähnlicher Häufigkeit erkennt, die Dokumente als derselben Klasse zugehörig klassifiziert. Dabei spielen die Reihenfolge der Wörter und der Kontext keine Rolle; wichtig sind das Vorhandensein und die Häufigkeit der Wörter.

In BoW wird jedes Wort als Koordinate in einem n-dimensionalen Raum dargestellt. Wenn wir den binären Parameter auf „True“ setzen, wird jedes Wort, das mehr als einmal vorkommt, trotzdem so behandelt, als ob es nur einmal vorkommt. Folglich sind die Koordinaten binär und nur mit 0en und 1en gefüllt.

### Beispieldaten
| Index | Text | Ausgabe |
|-------|--------------------------|--------|
| D1 | Personen folgen Mr.Beast | 1 |
| D2 | Mr.Beast folgt Mr.Beast | 1 |
| D3 | Personen schreiben Kommentar | 0 |
| D4 | Mr.Beast schreibt Kommentar | 0 |

### BoW mit `binary=False`

| Personen | Folgen | Mr.Beast | Schreiben | Kommentieren | Index |
|--------|--------|----------|-------|---------|-------|
| 1 | 1 | 1 | 0 | 0 | D1 |
| 0 | 1 | 2 | 0 | 0 | D2 |
| 1 | 0 | 0 | 1 | 1 | D3 |
| 0 | 0 | 1 | 1 | 1 | D4 |

### BoW mit `binary=True`

| Personen | Folgen | Mr.Beast | Schreiben | Kommentieren | Index |
|--------|--------|----------|-------|---------|-------|
| 1 | 1 | 1 | 0 | 0 | D1 |
| 0 | 1 | 1 | 0 | 0 | D2 |
| 1 | 0 | 0 | 1 | 1 | D3 |
| 0 | 0 | 1 | 1 | 1 | D4 |

### Filtern seltener Wörter

Um seltene Wörter zu entfernen, können Sie den Parameter „max_features“ verwenden, der das Vokabular auf die häufigsten Wörter beschränkt. Wenn Sie beispielsweise „max_features=1/2“ festlegen, werden alle Wörter ignoriert, deren Häufigkeit unter diesem Schwellenwert liegt.

### Vorteile von Bag of Words

- **Einfachheit und Intuitiv**: Leicht zu implementieren und zu verstehen.
- **Milderung des Out-of-Vocabulary (OOV)-Problems**: Da das Modell ein festes Vokabular verwendet, wird das OOV-Problem vermieden, das bei der One-Hot-Kodierung auftritt.

### Nachteile von Bag of Words

- **Spärlichkeit**: Bei einem großen Vokabular enthält die Darstellung viele Nullen, was möglicherweise zu Überanpassung führt.
- **Informationsverlust**: Ignoriert Wortreihenfolge und Kontext, was zu einem Verlust der semantischen Bedeutung führt.
- Beispiel: „Dies ist ein sehr guter Film“ und „Dies ist kein sehr guter Film“ können ähnlich dargestellt werden, was zu Problemen mit der Kosinusähnlichkeit führen kann.

### Kosinusähnlichkeit

Die Kosinusähnlichkeit misst den Kosinus des Winkels zwischen zwei Vektoren und liefert eine Metrik für ihre Ähnlichkeit. Die Formel lautet:

$$
\text{Kosinusähnlichkeit} = \cos(\theta) = \frac{A \cdot B}{\|A\| \|B\|}
$$

Mithilfe der Kosinusähnlichkeit können „Dies ist ein sehr guter Film“ und „Dies ist kein sehr guter Film“ aufgrund des geringen Winkels zwischen ihren Vektordarstellungen im BoW-Modell ähnlich erscheinen.

Mit BoW können Sie Textdaten schnell und effizient in numerische Merkmale für maschinelle Lernalgorithmen umwandeln. Beachten Sie jedoch die Einschränkungen und ziehen Sie fortgeschrittenere Techniken wie TF-IDF, Wort-Einbettungen oder Sprachmodelle in Betracht, um differenziertere Textinformationen zu erfassen.

In [7]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_cleaned['tweet_cleaned'])

vocab = vectorizer.get_feature_names_out()
print(vocab)
bow_df = pd.DataFrame(X.toarray(), columns=vocab)
bow_df 
#final_df = pd.concat([df_cleaned[['Index', 'Output']], bow_df], axis=1)

['aamp' 'aap' 'ab' ... 'zootopia' 'zoro' 'zzz']


,aamp,aap,ab,abandon,abasel,abba,abc,abe,ability,able,...,zionazi,zionism,zipline,zipper,zombie,zone,zoo,zootopia,zoro,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27644,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27645,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27646,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 2. Term Frequency-Inverse Document Frequency (TF-IDF)

TF-IDF behebt die Einschränkung der Gleichgewichtung von Begriffen in Bag of Words, indem Begriffen, die im Korpus selten, in einem bestimmten Dokument jedoch häufig vorkommen, höhere Gewichte zugewiesen werden. Dieser Ansatz ist entscheidend, um die Bedeutung von Begriffen in Dokumenten für Aufgaben wie Informationsabruf und Text Mining zu erfassen.

### Begriffshäufigkeit (TF)

Die Begriffshäufigkeit (TF) misst, wie häufig ein Begriff in einem Dokument im Verhältnis zur Gesamtzahl der Begriffe in diesem Dokument vorkommt. Sie wird wie folgt berechnet:

$$
\text{TF}(t, d) = \frac{\text{Anzahl der Vorkommen des Begriffs } t \text{ im Dokument } d}{\text{Gesamtzahl der Begriffe im Dokument } d}
$$

#### Beispiel:

Betrachten Sie ein Dokument: „Die Katze saß auf der Matte.“

- Häufigkeit der Begriffe für jeden Begriff:
- TF("the") = 2/6 = 1/3
- TF("cat") = 1/6
- TF("sat") = 1/6
- TF("on") = 1/6
- TF("mat") = 1/6

### Inverse Dokumenthäufigkeit (IDF)

Die inverse Dokumenthäufigkeit (IDF) misst, wie wichtig ein Begriff in allen Dokumenten eines Korpus ist. Sie wird wie folgt berechnet:

$$
\text{IDF}(t) = \log\left(\frac{\text{Gesamtzahl der Dokumente}}{\text{Anzahl der Dokumente, die den Begriff } t enthalten}\right)
$$

Sie quantifiziert die Seltenheit eines Begriffs in allen Dokumenten und gewichtet Begriffe, die in allen Dokumenten weniger häufig vorkommen, höher.

#### Beispiel:

Betrachten Sie ein Corpus mit 100 Dokumenten und der Begriff „Katze“ erscheint in 10 davon:

- IDF(„Katze“) = log(100/10) = log(10) ≈ 2,3026

### TF-IDF-Score

Der TF-IDF-Score für einen Begriff in einem Dokument ist das Produkt seiner Begriffshäufigkeit und seiner inversen Dokumenthäufigkeit:

$$
\text{TF-IDF}(t, d) = \text{TF}(t, d) \times \text{IDF}(t)
$$

### Beispielberechnung:

Berechnen wir TF-IDF für den gegebenen Beispieldatensatz:

#### Schritt 1: TF berechnen

Der Einfachheit halber nehmen wir binäres Zählen an:

| Index | Text | Begriffe | TF |
|-------|--------------------------|---------------------|-------------|
| D1 | Leute folgen Mr.Beast | Leute, folgen, mr.beast | 1/3 für jeden Begriff |
| D2 | Mr.Beast folgt Mr.Beast | mr.beast, folgen | 1/2 für mr.beast, 1/2 für folgen |
| D3 | Leute schreiben Kommentar | Leute, schreiben, kommentieren | 1/3 für jeden Begriff |
| D4 | Mr.Beast schreibt Kommentar | mr.beast, schreiben, kommentieren | 1/3 für mr.beast, 1/3 für schreiben, 1/3 für Kommentar |

#### Schritt 2: IDF berechnen

Angenommen, es gibt insgesamt 4 Dokumente im Korpus:

| Begriff | IDF-Formel | IDF (unter Annahme von Logarithmus zur Basis 2) |
|-----------|-------------------------------------|---------------------------|
| Leute | log(4/2) | log(2) = 1 |
| folgen | log(4/3) | log(4/3)=0,415 |
| Mr.Beast | log(4/3) | log(4/3)=0,415 |

### Vorteile und Nachteile

**Vorteile**:
- **Informationsabruf**: Wird in Suchmaschinen wie Google verwendet, um Dokumente nach Relevanz zu bewerten.
- **Begriffswichtigkeit**: Hebt Begriffe hervor, die in einem Dokument im Vergleich zu anderen deutlich oder wichtig sind.

**Nachteile**:
- **Sparcity**: Sparse-Vektoren können bei der Handhabung großer Vokabulare zu Problemen führen.
- **Out-of-Bag-Wörter**: Begriffe, die im Trainingskorpus nicht vorhanden sind, können zu null IDF und ineffektiver Darstellung führen.
- **Semantische Beziehung**: Erfasst keine semantischen Beziehungen zwischen Wörtern (z. B. können „schön“ und „umwerfend“ trotz unterschiedlicher Bedeutungen ähnliche TF-IDF haben).

In praktischen Implementierungen wie in sklearn wird häufig ein Glättungsterm „(1+TFIDF)“ hinzugefügt, um eine Division durch Null zu vermeiden und selbst für Begriffe, die in allen Dokumenten vorkommen, von Null verschiedene IDF-Werte sicherzustellen.

In [11]:
tfidf_vectorizer = TfidfVectorizer()

tweets = df_cleaned['tweet_cleaned']
labels = df_cleaned['label']

X_tfidf = tfidf_vectorizer.fit_transform(tweets)
X_tfidf
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_names

df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=feature_names)
df_tfidf
#df_final = pd.concat([df_cleaned[['Index', 'Output']], df_tfidf], axis=1)

# print("Final DataFrame with TF-IDF:")
# df_final

,aamp,aap,ab,abandon,abasel,abba,abc,abe,ability,able,...,zionazi,zionism,zipline,zipper,zombie,zone,zoo,zootopia,zoro,zzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27645,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### 3. Word Embeddings Word2Vec

#### 4. FastText